#1. spark 세팅

In [ ]:
!pip install pyspark
!pip install graphframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 199 kB 52.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=1a5d3cbec52d2a070479a7f626a7d8390d522ce64dc9570f10086ed01b082ace
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 7.5 MB/s 


In [ ]:
!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.0-spark2.4-s_2.11.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.0-spark2.4-s_2.11/graphframes-0.8.0-spark2.4-s_2.11.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file 
100   166  100   166    0     0   1844      0 --:--:-- --:--:-- --:--:--  1844
curl: (23) Failed writing body (0 != 166)


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq

Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u352-ga-1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u352-ga-1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u352-ga-1~18.04_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u352-ga-1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u352-ga-1~18.04) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/servertool to provide /usr/bin/servertool (servertool) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/tnameserv to provide /usr/bin/tnameserv (tnameserv) in auto mode
Setting up openjdk-8-jdk-headless:amd64 (8u352-ga-1~18

In [ ]:
import pyspark
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql import column
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql.functions import expr

In [ ]:
spark = SparkSession.builder.master("local[*]").config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12").getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
from pyspark.sql.types import *
from graphframes import *

#2. pyspark GraphFrame


## 2-1. 데이터 GraphFrame으로 불러오기

In [ ]:
def create_city_data():
  node_fields = [
      StructField("vertices", StringType(), True),
      StructField("id", IntegerType(), True),
      StructField("latitude", IntegerType(), True),
      StructField("longitude", IntegerType(), True)
  ]

  nodes = spark.read.csv("USA-road-d.NY.co",header=True, sep=' ', schema = StructType(node_fields))
  nodes = nodes.filter(nodes.vertices == 'v')
  nodes = nodes.drop(nodes.vertices)

  rel_fields = [
      StructField("arcs", StringType(), True),
      StructField("src", IntegerType(), True),
      StructField("dst", IntegerType(), True),
      StructField("cost", DoubleType(), True)
  ]

  relationships = spark.read.csv("USA-road-d.NY.gr", header=True, sep=' ', schema = StructType(rel_fields))
  relationships = relationships.filter(relationships.arcs == 'a')
  relationships = relationships.drop(relationships.arcs)

  return GraphFrame(nodes,relationships)

In [ ]:
gf = create_city_data()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:149: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  "DataFrame.sql_ctx is an internal property, and will be removed "


In [ ]:
gf.vertices.show()

+---+---------+---------+
| id| latitude|longitude|
+---+---------+---------+
|  1|-73530767| 41085396|
|  2|-73530538| 41086098|
|  3|-73519366| 41048796|
|  4|-73519377| 41048654|
|  5|-73524567| 41093796|
|  6|-73525490| 41093834|
|  7|-73531927| 41110484|
|  8|-73530106| 41110611|
|  9|-73529341| 41125895|
| 10|-73529746| 41127369|
| 11|-73530583| 41125051|
| 12|-73529763| 41085358|
| 13|-73529834| 41086062|
| 14|-73613384| 41065086|
| 15|-73615019| 41065131|
| 16|-73693133| 41058075|
| 17|-73694273| 41059296|
| 18|-73512230| 41287431|
| 19|-73511896| 41286556|
| 20|-73501634| 41286067|
+---+---------+---------+
only showing top 20 rows



In [ ]:
gf.edges.show()

+---+---+------+
|src|dst|  cost|
+---+---+------+
|  1|  2| 803.0|
|  2|  1| 803.0|
|  3|  4| 158.0|
|  4|  3| 158.0|
|  5|  6| 774.0|
|  6|  5| 774.0|
|  7|  8|1531.0|
|  8|  7|1531.0|
|  9| 10|1673.0|
| 10|  9|1673.0|
|  9| 11|1400.0|
| 11|  9|1400.0|
|  1| 12| 842.0|
| 12|  1| 842.0|
|  2| 13| 591.0|
| 13|  2| 591.0|
| 14| 15|1371.0|
| 15| 14|1371.0|
| 16| 17|1659.0|
| 17| 16|1659.0|
+---+---+------+
only showing top 20 rows



## 2-2. GraphFrame을 통해 위치정보(위도,경도) 찾기

In [ ]:
def node_position(id):
  a = gf.vertices.filter(f'id == {id}')

  return(a.first().latitude, a.first().longitude)

## 2-3. GraphFrame을 통해 인접노드 찾기

In [ ]:
def find_near_node(id):
  a = gf.edges.filter(f'src == {id}')
  return_data=[]

  for i in a.collect():
    return_data.append(i['dst'])

  return return_data

## 2-4. GraphFrame을 통해 Gscore 거리 찾기

In [ ]:
def get_g_score(src, dst):
  a = gf.edges.filter(f'src == {src} AND dst == {dst}')
  return(a.first()['cost'])

#3. heuristic function


##3-1. 유클리드거리

In [ ]:
def euclidean_distance(pt1, pt2):
  distance = 0

  for i in range(len(pt1)):
    distance += (pt1[i] - pt2[i]) ** 2

  return round(distance ** 0.5, 2)

#4. Node

##4-1. Node Class

In [ ]:
class Node:
    def __init__(self, parent=None, id=None):
        self.parent = parent
        self.id = id

        self.g_score = 0
        self.h_score = 0
        self.f_score = 0

    def __repr__(self):
        return f"{self.id}"

    def __eq__(self, other):
      return self.id == other.id

#5. aStar 알고리즘

In [ ]:
def a_star(origin, destination):
  # 출발 노드와 도착 노드 초기화
  origin_node = Node(None, origin)
  dest_node = Node(None, destination)

  open_list = []
  closed_list =[]

  open_list.append(origin_node)

  while open_list:

    current_node = open_list[0]
    current_idx = 0

    for index, item in enumerate(open_list):
            if item.f_score < current_node.f_score:
                current_node = item
                current_idx = index

    # open_list에서 제거하고 closed_list에 추가
    open_list.pop(current_idx)
    closed_list.append(current_node)

    # 현재 노드가 목적지면 current.id 추가하고
    # current의 부모로 이동
    if current_node == dest_node:
        path = []
        current = current_node
        while current is not None:
            path.append(current.id)
            current = current.parent
        return path[::-1]  # reverse

    children = []
    # 인접한 노드 찾기
    near_nodes = find_near_node(current_node.id)

    for nodeid in near_nodes:
      new_node = Node(current_node, nodeid)
      children.append(new_node)

    # 자식들 모두 loop
    for child in children:
      # 자식이 closedList에 있으면 continue
      if child in closed_list:
        continue

      # f_score, g_score, h_score값 업데이트
      child.g_score = current_node.g_score + get_g_score(current_node.id ,child.id)
      child.h_score = euclidean_distance(node_position(child.id), node_position(dest_node.id))
      child.f_score = child.g_score + child.h_score

      # 자식이 open_list에 있으고, g_score 값이 더 크면 continue
      if len([open_node for open_node in open_list
              if child == open_node and child.g_score > open_node.g_score]) > 0:
          continue

      open_list.append(child)


In [ ]:
# 정답 예시
# 출발노드 : 1,
# 도착노드 : 10
# 약 3분 소요
a_star(1, 10)

[1,
 1363,
 1358,
 1355,
 1274,
 1183,
 1181,
 1182,
 1179,
 1177,
 1176,
 1178,
 1154,
 1153,
 1072,
 1148,
 7,
 1147,
 598,
 596,
 586,
 587,
 591,
 590,
 592,
 588,
 594,
 603,
 9,
 10]